# MWS Secondary Target Selection

## MWS_SPECIAL_RRLYR_SV

Maintainers:
  - Sergey Koposov (selection/catalog)
  - Andrew Cooper (secondary target code)

In [1]:
ls ../raw

rrlyrae_gaia_source_id.fits


We include all the stars from Gaia that were classified as RR Lyrae with mean G magnitudes between 14 and 19. Specifically that includes

1. All the stars from Gaia vari_rrlyrae table
2. All the stars from vari_classifier_result with RR* class

The selection criteria are:

```
WITH x as (SELECT vari_classifier_result.source_id
        FROM gaia_dr2.vari_classifier_result
        WHERE vari_classifier_result.best_class_name::text ~~ 'RR%'::text
    UNION
    SELECT vari_rrlyrae.source_id
        FROM gaia_dr2.vari_rrlyrae)
SELECT g.* FROM gaia_dr2.gaia_source as g, x
       where g.source_id = x.source_id
       and phot_g_mean_mag between 14 and 19;
```

In [2]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/code'

In [3]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [4]:
target_class   = 'MWS_SPECIAL_RRLYR_SV'
input_filename = 'rrlyrae_gaia_source_id.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [5]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [6]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_RRLYR_SV/1/raw/rrlyrae_gaia_source_id.fits
Read 137249 rows


In [9]:
print('First 3 rows:')
print(d[0:3])
print(d.colnames)

First 3 rows:
     source_id     
-------------------
6026375677963210240
5393238919477110528
 834731365039417856
['source_id']


In [8]:
# Match on Gaia source ID

In [25]:
import re
import glob

In [22]:
gaia_sources = glob.glob('/global/project/projectdirs/cosmo/staging/gaia/dr2/gaia-source/GaiaSource_*.fits')

In [97]:
pat = 'GaiaSource_(:?[0-9]+)_(:?[0-9]+).fits'
lows, highs = list(), list()
for fn in gaia_sources:
    m   = re.match(pat,os.path.basename(fn))
    low, high = m.groups()
    lows.append(int(low))
    highs.append(int(high))
lows  = np.array(lows)
highs = np.array(highs)

In [98]:
np.argmin(lows), np.argmax(lows)

(19761, 6013)

In [99]:
np.argmin(highs), np.argmax(highs)

(19761, 6013)

In [100]:
lows[0:3], highs[0:3]

(array([6003463230033802752, 4112108822858560896, 4269275660632871040]),
 array([6003509997942660480, 4112131504619931520, 4269297311621972864]))

In [101]:
ifile = np.arange(0,len(lows))
assert(np.all(np.argsort(lows) == np.argsort(highs)))
s     = np.argsort(lows)
ifile = ifile[s]
lows  = lows[s]
highs = highs[s]

In [102]:
lows[0:3], highs[0:3]

(array([     4295806720, 478047042392960, 933966408964096]),
 array([ 477948256339712,  933932049175296, 1387484890373248]))

In [143]:
low_edge  = np.searchsorted(lows,d['source_id'],side='left')
high_edge = np.searchsorted(highs,d['source_id'],side='right')+1

In [146]:
(high_edge-low_edge > 0).sum()

22

In [144]:
assert(np.all(high_edge - low_edge == 0))

AssertionError: 

In [112]:
print(ifile[low_edge[0:3]])

[40056 61041 37517]


In [113]:
low_edge[0],high_edge[0], ifile[low_edge[0]], ifile[high_edge[0]]

(54770, 54770, 40056, 40056)

In [114]:
print(gaia_sources[ifile[low_edge[0]]])

/global/project/projectdirs/cosmo/staging/gaia/dr2/gaia-source/GaiaSource_6026391792684395008_6026414543209870208.fits


In [107]:
d['source_id'][0] >= low_edge[0], d['source_id'][0] < high_edge[0]

(True, False)

In [90]:
d['source_id'][0]-6026391792684395008,6026414543209870208-d['source_id'][0]

(-16114721184768, 38865246659968)

In [18]:
t = Table.read('/global/project/projectdirs/cosmo/staging/gaia/dr2/gaia-source/GaiaSource_1000172165251650944_1000424567594791808.fits')

In [21]:
t[np.argsort(t['source_id'])][-3:]

solution_id,designation,source_id,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,astrometric_weight_al,astrometric_pseudo_colour,astrometric_pseudo_colour_error,mean_varpi_factor_al,astrometric_matched_observations,visibility_periods_used,astrometric_sigma5d_max,frame_rotator_object_type,matched_observations,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_nb_transits,rv_template_teff,rv_template_logg,rv_template_fe_h,phot_variable_flag,l,b,ecl_lon,ecl_lat,priam_flags,teff_val,teff_percentile_lower,teff_percentile_upper,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,flame_flags,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper
int64,bytes28,int64,int64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,int16,float32,float32,float32,float32,float32,float32,bool,float32,float32,float32,float32,int16,int16,float32,int16,int16,bool,int16,float32,float32,float32,float32,int16,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,int16,float32,float32,float32,bool,float64,float64,float64,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32
1635721458409799680,Gaia DR2 1000424537530485888,1000424537530485888,1243536732,2015.5,105.42970382113742,0.03833766,56.736814651510315,0.035825465,0.58066165,0.05678236,10.226092,3.169589,0.07611264,-6.4370375,0.06194767,0.18521082,-0.20333645,-0.06818303,-0.14529514,-0.2780403,-0.23191848,0.07528074,0.56536627,0.23448981,0.26072592,204,0.0,202,2.0,-2.4255574,152.10799,0.0,0.0,31.0,False,8.954648,1.5932878,0.012166413,-0.03546068,23,12,0.08118331,0,25,False,220,7799.382,4.6971893,1660.4359,15.958216,21,4152.258,24.37694,170.33548,16.305677,19,5327.918,12.031983,442.81293,15.445526,1.2155035,0,0.8601513,0.3474617,0.5126896,nan,nan,0,nan,nan,nan,False,159.71247091767341,23.776231117904715,100.1186479577399,33.838261009774484,-31071,5764.0,5579.0,5903.1997,0.0342,0.017,0.2091,0.009,0.0069,0.1001,3503,0.958838,0.9141516,1.0234826,0.9142838,0.7211792,1.1073885
1635721458409799680,Gaia DR2 1000424537530635008,1000424537530635008,902341180,2015.5,105.44138839754099,0.7017483,56.72564544238555,0.7121367,0.08306585,1.1894765,0.06983396,1.2567902,1.5981851,-4.287342,0.97887003,0.38510612,-0.36753878,-0.40891877,-0.21762261,-0.6531118,-0.5284575,-0.23408842,0.72934073,0.31477088,0.40853322,142,0.0,141,1.0,-2.0420485,104.46501,0.0,0.0,31.0,False,0.043696277,1.492231,0.21210697,-0.0698243,16,10,1.6914191,0,17,False,149,148.92442,1.2158561,122.48524,20.25595,11,87.39034,8.013672,10.905156,20.49773,13,102.62113,10.658108,9.628457,19.733828,1.2758919,0,0.76390266,0.24177933,0.52212334,nan,nan,0,nan,nan,nan,False,159.72598764026057,23.779813811291746,100.12801523297973,33.827999831801925,-1,nan,nan,nan,nan,nan,nan,nan,nan,nan,-1,nan,nan,nan,nan,nan,nan
16357214584

In [14]:
!cat /global/project/projectdirs/cosmo/staging/gaia/dr2/README

Gaia DR2 FITS files

csv-files.txt: a list of CSV files in the Gaia-DR2 release.

mkdir CSV-gaia-source
python get.py

-> reads csv-files.txt, downloads all files to CSV-gaia-source

mkdir gaia-source-orig
python conv.py

-> reads CSV files, produces FITS files

mkdir gaia-source
python conv2.py

-> reads FITS files, fixes some issues.



In [52]:
BRIGHT_TIME_BRIGHT_LIMIT = 16.0

print('Mag range: {:5.2f} < G < {:5.2f}'.format(d['phot_g_mean_mag'].min() ,d['phot_g_mean_mag'].max()))
print('Targets brighter than G = 16: {}/{} ({:3.2%})'.format((d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum(),
                                                   len(d),
                                                   (d['phot_g_mean_mag'] < BRIGHT_TIME_BRIGHT_LIMIT).sum()/len(d)
                                                  ))

bright_time_targets = d['phot_g_mean_mag'] > BRIGHT_TIME_BRIGHT_LIMIT

Mag range: 10.67 < G < 20.88
Targets brighter than G = 16: 75258/291411 (25.83%)


Adjust to secondary target data model:

In [53]:
output = d['ra','dec','source_id'][bright_time_targets]
output.rename_columns(['ra','dec','source_id'],['RA','DEC','REF_ID'])
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [54]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

In [55]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
328.530788027781,2.53707028993252,2693964742599255040,False,0.0,0.0,G,2015.5
328.59327920521,2.44939161551723,2693916226648580224,False,0.0,0.0,G,2015.5
328.591951469507,2.36698433447594,2693912374062424192,False,0.0,0.0,G,2015.5


Write output:

In [56]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 216153 rows of output to /Users/apcooper/astro/data/desi/secondary/MWS_SPECIAL_DDOGIANTS_SV/1/indata/MWS_SPECIAL_DDOGIANTS_SV.fits
